Trabalho realizado por: Bárbara Freixo, PG49169

Esta implementação define uma classe Dataset para manipular dados armazenados em arquivos CSV/TSV. Ela fornece funcionalidades para ler e escrever arquivos CSV/TSV, descrever o conjunto de dados e encontrar e substituir missing values. Temos então os seguintes métodos:

* __init__: Inicializa a classe Dataset com a matriz de entrada X, o vetor de saída y, os nomes das características (features), o nome da variável alvo (label) e um dicionário para armazenar os valores discretos das características discretas.
* get_X: Retorna a matriz X.
* get_y: Retorna o vetor y.
* get_feature_names: Retorna os nomes das características.
* get_label_name: Retorna o nome da variável alvo (label).
* set_X: Define a matriz X.
* set_y: Define o vetor y.
* set_feature_names: Define os nomes das características.
* set_label_name: Define o nome da variável alvo (label).
* set_discrete_values: Define os valores discretos.
* get_discrete_values: Retorna os valores discretos.
* read_csv: Lê um arquivo CSV/TSV e armazena os dados na matriz X e no vetor y. Os nomes das características e da variável alvo são armazenados, se houver um cabeçalho no arquivo.

* write_csv: Escreve os dados do objeto Dataset num arquivo CSV.

* describe: Calcula e retorna a média, desvio padrão, valor mínimo e valor máximo de cada característica na matriz X.

* find_missing_values: Retorna uma matriz booleana indicando a posição dos missing values na matriz X.

* count_missing_values: Conta e retorna o número de missing values em cada coluna da matriz X.

* replace_missing_values: Substitui os missing values na matriz X por uma constante. A constante pode ser a média, mediana, moda ou um valor específico. Para características discretas, a moda é usada como padrão.

In [3]:
import numpy as np
import csv
from scipy.stats import mode

class Dataset:
    def __init__(self, X=None, y=None, feature_names=None, label_name=None):
        """
        Inicializa a classe Dataset com a matriz de entrada X, o vetor de saída y,
        os nomes das características (features), o nome da variável alvo (label) e
        um dicionário para armazenar os valores discretos das características discretas.
        """
        self.X = X
        self.y = y
        self.feature_names = feature_names
        self.label_name = label_name
        self.discrete_values = {}

    def get_X(self):
        """Retorna a matriz X."""
        return self.X

    def get_y(self):
        """Retorna o vetor y."""
        return self.y

    def get_feature_names(self):
        """Retorna os nomes das características."""
        return self.feature_names

    def get_label_name(self):
        """Retorna o nome da variável alvo (label)."""
        return self.label_name

    def set_X(self, X):
        """Define a matriz X."""
        self.X = X

    def set_y(self, y):
        """Define o vetor y."""
        self.y = y

    def set_feature_names(self, feature_names):
        """Define os nomes das características."""
        self.feature_names = feature_names

    def set_label_name(self, label_name):
        """Define o nome da variável alvo (label)."""
        self.label_name = label_name

    def set_discrete_values(self, discrete_values):
        """Define os valores discretos."""
        self.discrete_values = discrete_values

    def get_discrete_values(self):
        """Retorna os valores discretos."""
        return self.discrete_values

    def read_csv(self, filepath, delimiter=',', has_header=True):
        """
        Lê um arquivo CSV/TSV e armazena os dados na matriz X e no vetor y.
        Os nomes das características e da variável alvo são armazenados, se houver
        um cabeçalho no arquivo.
        """
        with open(filepath, 'r') as file:
            reader = csv.reader(file, delimiter=delimiter)
            if has_header:
                header = next(reader)
                self.feature_names = header[:-1]
                self.label_name = header[-1]
            data = list(reader)
        data = np.array(data, dtype=object)
        self.X = data[:, :-1].astype(np.float64)
        self.y = np.round(data[:, -1].astype(np.float64)).astype(np.int64)

    def write_csv(self, filepath, delimiter=','):
        """
        Escreve os dados do objeto Dataset num arquivo CSV/TSV.
        """
        data = np.column_stack((self.X, self.y))
        with open(filepath, 'w') as file:
            writer = csv.writer(file, delimiter=delimiter)
            if self.feature_names is not None and self.label_name is not None:
                writer.writerow(self.feature_names + [self.label_name])
            writer.writerows(data)

    def describe(self):
        """
        Calcula e retorna a média, desvio padrão, valor mínimo e valor máximo de cada
        característica na matriz X.
        """
        means = np.mean(self.X.astype(np.float64), axis=0)
        stds = np.std(self.X.astype(np.float64), axis=0)
        min_values = np.min(self.X.astype(np.float64), axis=0)
        max_values = np.max(self.X.astype(np.float64), axis=0)

        return means, stds, min_values, max_values

    def find_missing_values(self):
        """
        Retorna uma matriz booleana indicando a posição dos missing values
        na matriz X.
        """
        return np.isnan(self.X.astype(np.float64))

    def count_missing_values(self):
        """
        Conta e retorna o número de missing values em cada coluna da matriz X.
        """
        return np.sum(np.isnan(self.X.astype(np.float64)), axis=0)

    def replace_missing_values(self, constant='mean'):
        """
        Substitui os missing values na matriz X por uma constante. A constante
        pode ser a média, mediana, moda ou um valor específico. Para características
        discretas, a moda é usada como padrão.
        """
        missing = self.find_missing_values()
        for i in range(self.X.shape[1]):
            if constant == 'mean':
                if i in self.discrete_values:
                    replacement = mode(self.X[:, i])[0][0]
                else:
                    replacement = np.nanmean(self.X[:, i].astype(np.float64))
            elif constant == 'median':
                replacement = np.nanmedian(self.X[:, i].astype(np.float64))
            elif constant == 'mode':
                replacement = mode(self.X[:, i])[0][0]
            else:
                replacement = constant
            self.X[:, i][missing[:, i]] = replacement



# Testes e exemplos para o algoritmo implementado

## Exemplo de uso do código

Esta implementação demonstra como manipular e analisar um conjunto de dados utilizando a classe Dataset, além de mostrar como salvar e carregar os dados em formato CSV.

In [4]:
from sklearn.datasets import load_wine

# Carrega o conjunto de dados Wine
wine = load_wine()

# Cria uma instância da classe Dataset e preenche os atributos com os dados do Wine
dataset = Dataset(X=wine.data, y=wine.target, feature_names=wine.feature_names, label_name='wine_class')

# Imprime as variáveis independentes (X), a variável dependente (y), os nomes das características e o nome do rótulo
print("X:", dataset.get_X())
print("y:", dataset.get_y())
print("Feature names:", dataset.get_feature_names())
print("Label name:", dataset.get_label_name())

# Calcula e imprime as estatísticas do conjunto de dados
means, stds, min_values, max_values = dataset.describe()
print("Means:", means)
print("Standard deviations:", stds)
print("Minimum values:", min_values)
print("Maximum values:", max_values)

# Identifica e conta os missing values
missing_values = dataset.find_missing_values()
missing_count = dataset.count_missing_values()
print("Missing values:", missing_values)
print("Missing count:", missing_count)

# Escreve o conjunto de dados num arquivo CSV
dataset.write_csv("wine.csv")

# Remove o arquivo CSV após a execução do exemplo
import os
os.remove("wine.csv")

X: [[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]]
y: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Feature names: ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']

## Testes "Unittest"

Foi realizado um teste utilizando o unittest para testar a implementação do Dataset. Temos então o seguinte teste:

Estes testes cobrem funções como getters e setters, leitura e escrita de arquivos CSV, descrição e manipulação de missing values.

In [6]:
import unittest
import numpy as np
from sklearn.datasets import load_iris

class TestDataset(unittest.TestCase):
    """
    Classe de teste para a classe Dataset.
    """

    def setUp(self):
        """
        Carrega o conjunto de dados Iris e cria uma instância da classe Dataset
        com os dados e rótulos do conjunto de dados Iris.
        """
        self.iris = load_iris()
        self.dataset = Dataset(X=self.iris.data, y=self.iris.target,
                               feature_names=self.iris.feature_names,
                               label_name='species')

    def test_get_X(self):
        """Testa o método get_X."""
        self.assertTrue(np.array_equal(self.dataset.get_X(), self.iris.data))

    def test_get_y(self):
        """Testa o método get_y."""
        self.assertTrue(np.array_equal(self.dataset.get_y(), self.iris.target))

    def test_get_feature_names(self):
        """Testa o método get_feature_names."""
        self.assertEqual(self.dataset.get_feature_names(), self.iris.feature_names)

    def test_get_label_name(self):
        """Testa o método get_label_name."""
        self.assertEqual(self.dataset.get_label_name(), 'species')

    def test_describe(self):
        """Testa o método describe."""
        means, stds, min_values, max_values = self.dataset.describe()

        self.assertTrue(np.allclose(means, np.mean(self.iris.data, axis=0)))
        self.assertTrue(np.allclose(stds, np.std(self.iris.data, axis=0)))
        self.assertTrue(np.allclose(min_values, np.min(self.iris.data, axis=0)))
        self.assertTrue(np.allclose(max_values, np.max(self.iris.data, axis=0)))

    def test_missing_values(self):
        """Testa os métodos find_missing_values e count_missing_values."""
        missing_values = self.dataset.find_missing_values()
        missing_count = self.dataset.count_missing_values()

        self.assertEqual(missing_values.shape, self.iris.data.shape)
        self.assertEqual(missing_count.shape, (self.iris.data.shape[1],))
        self.assertEqual(np.sum(missing_values), 0)
        self.assertEqual(np.sum(missing_count), 0)

    def test_replace_missing_values(self):
        """
        Testa o método replace_missing_values, adicionando alguns missing values
        ao conjunto de dados e verifica se eles são substituídos corretamente.
        """

        data_with_missing = self.iris.data.copy()
        data_with_missing[0, 0] = np.nan
        data_with_missing[1, 1] = np.nan
        self.dataset.set_X(data_with_missing)

        self.dataset.replace_missing_values(constant='mean')
        means = np.nanmean(data_with_missing, axis=0)
        self.assertTrue(np.allclose(self.dataset.get_X()[0, 0], means[0]))
        self.assertTrue(np.allclose(self.dataset.get_X()[1, 1], means[1]))

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.039s

OK
